In [34]:
from keras.layers import Input, Dense
from keras.models import Model

In [35]:
from keras.datasets import mnist
import numpy as np

In [36]:
(x_train, _), (x_test, _) = mnist.load_data()

In [37]:
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

In [38]:
input_img = Input(shape=(784,))
encoded = Dense(128, activation='relu')(input_img)
encoded = Dense(64, activation='relu')(encoded)
encoded = Dense(32, activation='relu')(encoded)

decoded = Dense(64, activation='relu')(encoded)
decoded = Dense(128, activation='relu')(decoded)
decoded = Dense(784, activation='sigmoid')(decoded)

autoencoder = Model(input_img, decoded)

In [39]:
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')
autoencoder.fit(x_train, x_train,
                epochs=100,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/100
60000/60000 [==============================] - 4s 61us/step - loss: 0.3605 - val_loss: 0.2652
Epoch 2/100
60000/60000 [==============================] - 3s 56us/step - loss: 0.2579 - val_loss: 0.2517
Epoch 3/100
60000/60000 [==============================] - 3s 58us/step - loss: 0.2452 - val_loss: 0.2393
Epoch 4/100
60000/60000 [==============================] - 3s 57us/step - loss: 0.2351 - val_loss: 0.2278
Epoch 5/100
60000/60000 [==============================] - 4s 59us/step - loss: 0.2223 - val_loss: 0.2118
Epoch 6/100
60000/60000 [==============================] - 4s 61us/step - loss: 0.2027 - val_loss: 0.1958
Epoch 7/100
60000/60000 [==============================] - 4s 59us/step - loss: 0.1874 - val_loss: 0.1812
Epoch 8/100
60000/60000 [==============================] - 4s 59us/step - loss: 0.1787 - val_loss: 0.1745
Epoch 9/100
60000/60000 [==============================] - 4s 69us/step - loss: 0.1729 - val_loss: 0.1

60000/60000 [==============================] - 4s 69us/step - loss: 0.1043 - val_loss: 0.1028
Epoch 78/100
60000/60000 [==============================] - 4s 63us/step - loss: 0.1041 - val_loss: 0.1020
Epoch 79/100
60000/60000 [==============================] - 4s 61us/step - loss: 0.1039 - val_loss: 0.1032
Epoch 80/100
60000/60000 [==============================] - 4s 60us/step - loss: 0.1036 - val_loss: 0.1027
Epoch 81/100
60000/60000 [==============================] - 4s 62us/step - loss: 0.1034 - val_loss: 0.1042
Epoch 82/100
60000/60000 [==============================] - 4s 62us/step - loss: 0.1031 - val_loss: 0.1018
Epoch 83/100
60000/60000 [==============================] - 4s 66us/step - loss: 0.1030 - val_loss: 0.1025
Epoch 84/100
60000/60000 [==============================] - 4s 62us/step - loss: 0.1029 - val_loss: 0.1037
Epoch 85/100
60000/60000 [==============================] - 4s 61us/step - loss: 0.1024 - val_loss: 0.1026
Epoch 86/100
60000/60000 [========================

In [25]:
autoencoder

In [41]:
import torch as t
import torch.nn as nn
import torch.optim as optim

In [42]:
class InfoVAE(nn.Module):
  def __init__(self, H, W, C_in, C_h1, C_h2, D_h, D_z):
    super(InfoVAE, self).__init__()

    self.enc = enc = nn.Sequential(
      nn.Conv2d(C_in, C_h1, 4, 2, padding=1), # 28 -> 14
      nn.LeakyReLU(inplace=True),
      nn.Conv2d(C_h1, C_h2, 4, 2, padding=1), # 14 -> 7
      nn.LeakyReLU(inplace=True),
      Flatten(),
      nn.Linear(C_h2*H/4*W/4, D_h),
      nn.LeakyReLU(inplace=True),
      nn.Linear(D_h, D_z))
    self.dec = dec = nn.Sequential(
      nn.Linear(D_z, D_h),
      nn.ReLU(inplace=True),
      nn.Linear(D_h, C_h2*H/4*W/4),
      nn.ReLU(inplace=True),
      Reshape(C_h2, H/4, W/4),
      nn.ConvTranspose2d(C_h2, C_h1, 4, 2, padding=1),
      nn.ReLU(inplace=True),
      nn.ConvTranspose2d(C_h1, C_in, 4, 2, padding=1),
      nn.Sigmoid())
    self.opt = optim.Adam(it.chain(enc.parameters(), dec.parameters()))

  def preprocess(self, x):
    return channels_first(x)

  def kernel(self, a, b):
    N_a, D = a.size()
    N_b = b.size(0)

    a = a.unsqueeze(1).expand(N_a, N_b, D)
    b = b.unsqueeze(0).expand(N_a, N_b, D)

    return a.sub(b).pow_(2).mean(dim=2).div_(D).mul_(-1).exp_()

  def mmd_loss(self, y_prd, y):
    y_prd_k = self.kernel(y_prd, y_prd)
    y_k = self.kernel(y, y)
    y_prd_y_k = self.kernel(y_prd, y)

    # return y_k.mean() + y_prd_k.mean() - 2*y_prd_y_k.mean()
    return y_prd_y_k.mul(-2).add_(y_prd_k).add_(y_k).mean()

  def forward(self, x, y):
    z = self.enc(x)
    y_prd = self.dec(z)

    mmd = self.mmd_loss(z, t.randn_like(z))
    nll = y_prd.sub(y).pow_(2).mean()

    return (y_prd, mmd.add_(nll))

In [53]:
x = x_train.reshape((-1, 1, 28, 28))

In [54]:
x = t.tensor(x)

In [55]:
x.shape

torch.Size([60000, 1, 28, 28])

In [48]:
def channels_first(x):
  return x.permute(0, 3, 1, 2) # NHWC -> NCHW

In [56]:
N, H, W, C = x.size()
D = H * W * C

In [57]:
x = channels_first(x)

In [58]:
vae = InfoVAE(H, W, C, 64, 128, 1024, 2)

NameError: global name 'Flatten' is not defined

In [59]:
import os

In [60]:
os.system("quit /Applications/Safari.app/")

32512

In [61]:
os.system("pstree")

0

In [62]:
import subprocess

In [63]:
subprocess.Popen("pstree", stdout=subprocess.PIPE, shell=True)

In [64]:
p = _

In [65]:
p.stdout

<open file '<fdopen>', mode 'rb' at 0x118a025d0>

In [66]:
p.stdout.read

<function read>

In [68]:
p.stdout.read().index("Safari")

ValueError: substring not found